# Tarea 2. Minería de texto básica

Guillermo Segura Gómez

## Bolsas de Palabras, Bigramas y Emociones

Representa los documentos y clasifica con SVM similar a la Práctica 3, pero con diferentes
pesados de términos.

1. Evalué BoW con pesado binario.
2. Evalué BoW con pesado frecuencia.
3. Evalué BoW con pesado tfidf.
4. Evalué BoW con pesado binario normalizado l2 (no use sklearn).
5. Evalué BoW con pesado frecuencia normalizado l2 (no use sklearn).
6. Evalué BoW con pesado tfidf normalizado l2 (no use sklearn).
7. Ponga una tabla comparativa a modo de resumen con las seis entradas anteriores.
8. De las configuraciones anteriores elija la mejor y evalúela con más y menos términos
(e.g., 1000 y 7000). Ponga una tabla dónde compare las tres configuraciones.
9. Utilice el recurso léxico del Consejo Nacional de Investigación de Canadá llamado
"EmoLex" (https://www.saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm) para
construir una "Bolsa de Emociones" de los Tweets de agresividad (Debe usar EmoLex
en Español). Para esto, una estrategia sencilla sería enmascarar cada palabra con su
emoción, y después construir la Bolsa de Emociones (BoE).
10. Evalúa tú BoE clasificando con SVM. Ponga una tabla comparativa a modo de resumen
con los tres pesados, normalize cada uno si lo cree conveniente.

---

Para las bolsas de palabras que vamos a crear, vamos a utilizar el corpus [MEX-A3T](https://sites.google.com/view/mex-a3t/home?authuser=0) que evalua la agresividad en un conjunto de tweets. Tenemos dos conjuntos de datos, los de entrenamiento y los de prueba o validación. Cada conjunto contiene dos archivos, uno es una serie de documentos donde cada documento es un tweet; el segundo archivo consiste en las etiquetas, donde 0 corresponde a si es agresivo el tweet y 1 si no lo es. 

La bolsa de palabras es un modelo que simplifica el contenido textual al considerar solo la ocurrencia de palabras, ignorando su orden y contexto. En este modelo, un texto se representa como un vector, donde cada dimensión (columna) corresponde a una palabra del vocabulario de todos los textos considerados (filas), el valor en cada dimensión depende del esquema de pesado que se elija. 

Para construir una bolsa de palabras de manera manual, lo que tenemos que hacer es primero, construir el vocabulario completo del corpus. Del vocabulario es posible descartar las palabras menos frecuentes para tener una matriz de dimensionalidad mas baja con la cual trabajar. Una vez construido el vocabulario, podemos comenzar con la implementación de la bolsa de palabras en una matriz, respetando que cada fila tiene un documento y cada columna corresponde a un término del vocabulario. Según el esquema de pesado se va llenando la matriz. Es bástante útil construir un diccionario con la siguiente estructura {'palabra':'valor'}, de esta manera sabemos exactamente el lugar o columna donde corresponde la palabra encontrada. Además es de fácil y rápido acceso ya que un diccionario en python es una tabla hash. 

De esta manera comenzamos importando los documentos. Para procesarlos, construir el diccionario y las diferentes funciones de bolsas de palabras. 


In [1]:
# Función que extrae el texto de dos archivos. Uno el de los documentos, otro el de las etiquetas
def get_text_from_file(path_corpus, path_truth):

    tr_text = []
    tr_labels = []

    with open(path_corpus, "r") as f_corpus, open(path_truth, "r") as f_truth:
        for tweet in f_corpus:
            tr_text += [tweet]
        for label in f_truth:
            tr_labels += [label]

    return tr_text, tr_labels

In [2]:
path_text = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_train.txt"
path_labels = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_train_labels.txt"

path_text_val = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_val.txt"
path_labels_val = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_val_labels.txt"

tr_text, tr_labels = get_text_from_file(path_text, path_labels) # Importamos los datos de entrenamiento
val_text, val_labels = get_text_from_file(path_text_val, path_labels_val) # Importamos los datos de test o validación

### Construcción del vocabulario
Ahora construimos el vocabulario de la bow. Necesitamos tokenizar el total del corpus y guardarlo en una lista para poder construir el vocabulario total. Utilizamos el método de **TweetTokenizer** de la clase *tokenize* de la librería nltk.

In [3]:
import nltk
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer() # Inicializar tokenizer

corpus_palabras = []

for doc in tr_text:
    corpus_palabras += tokenizer.tokenize(doc)

fdist = nltk.FreqDist(corpus_palabras)

print(f"El tamaño del corpus es:", len(corpus_palabras))
print(f"El tamaño del vocabulario es:", len(fdist))

El tamaño del corpus es: 97473
El tamaño del vocabulario es: 15194


Ahora necesitamos ordenar las frecuencias del vocabulario para poder trabajar con los tokens mas comunes. Tomamos los cincomil términos mas frecuentes del vocabulario, ya que 15194 es bastante para trabajar, además de que los términos menos frecuentes no tienen tanta contribución. 

In [4]:
# Función que ordena un arreglo
def  SortFrecuency(freqdist):
    # List comprenhension
    aux = [(freqdist[key], key) for key in freqdist]
    aux.sort() # Ordena la lista
    aux.reverse() # Cambiar el orden

    return aux

voc = SortFrecuency(fdist)
voc = voc[:5000]
voc[:5]

[(3016, ','), (2915, 'de'), (2829, 'que'), (2604, '.'), (2031, 'la')]

Ahora necesitamos realizar un diccionario al que el valor de acceso sea la palabra. Esto con los fines explicados anteriormente.

In [5]:
dict_indices = dict()
count = 0

for weight, word in voc:
    dict_indices[word] = count
    count += 1

dict_indices["presidente"] # Ejemplo de uso

992

### 1. Evalué BoW con pesado binario.

La función que construye la bolsa de palabras con pesado binario únicamente, coloca 1 en la casilla donde se encontro la palabra. Se queda en 0 si no lo encuentra.

In [6]:
import numpy as np # Manejo de vectores

def build_bow_binary(tr_text, vocabulary, dict_indices):
    # Construcción de matriz para la bolsa de palabras
    # En cada fila vemos los documentos que estamos procesando
    # En las columnas el tamaño del vocabulario que estamos creando
    BOW = np.zeros((len(tr_text),len(vocabulary)), dtype = int)

    for idx, tr in enumerate(tr_text):

        # Cada documento tr lo tokenizamos
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))

        # Contamos cada palabra
        for word in fdist_doc:
            # Nos aseguramos que las palabras estan en el diccionario final
            if word in dict_indices:
                BOW[idx, dict_indices[word]] = 1 # Esquema de pesado binario

    return BOW

In [7]:
BOW_tr = build_bow_binary(tr_text, voc, dict_indices)
BOW_tr[:10][:10]

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

### 2. Evalué BoW con pesado frecuencia.

Para cambiar el esquema de pesado, ahora contamos el número de palabras en cada documento. 

In [8]:
def build_bow_frecuency(tr_text, vocabulary, dict_indices):
    # Construcción de matriz para la bolsa de palabras
    BOW = np.zeros((len(tr_text), len(vocabulary)), dtype=int)

    # Iteramos sobre cada documento en tr_text
    for idx, tr in enumerate(tr_text):

        # Cada documento tr lo tokenizamos
        tokens = nltk.FreqDist(tokenizer.tokenize(tr))

        # Calculamos la frecuencia de cada palabra en el documento
        fdist_doc = nltk.FreqDist(tokens)

        # Iteramos sobre cada palabra y su frecuencia en el documento
        for word, freq in fdist_doc.items():
            # Nos aseguramos que las palabras estan en el diccionario final
            if word in dict_indices:
                BOW[idx, dict_indices[word]] = freq  # Esquema de pesado de frecuencias

    return BOW

In [9]:
BOW_tr = build_bow_frecuency(tr_text, voc, dict_indices)
BOW_tr[:10][:10]

array([[0, 2, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

### 3. Evalué BoW con pesado tfidf.

**TF-IDF** es una técnica de pesado, en este caso para la bolsa de palabras, consiste en dos partes:

- **TF (Term Frequency)**: Mide cuán frecuente es una palabra en un documento. Hay varias maneras de calcularlo, pero una forma común es simplemente contar el número de veces que la palabra aparece en el documento y dividirlo por el número total de palabras en el documento. Esto normaliza la frecuencia de la palabra.

- **IDF (Inverse Document Frequency)**: Mide la importancia de la palabra en todo el conjunto de documentos. Se calcula tomando el logaritmo del número total de documentos dividido por el número de documentos que contienen la palabra. Esto da más peso a las palabras que son raras en todo el conjunto de documentos.

El valor TF-IDF es simplemente el producto de TF e IDF. Este valor será alto para palabras que aparecen frecuentemente en un documento, pero no en muchos documentos, lo que significa que dichas palabras son potencialmente más relevantes para el documento.

Para el esquema TFIDF necesitamos calcular primero el TF y posterior y el IDF.

In [10]:
def build_bow_tfidf(tr_text, vocabulary, dict_indices):
    # Construcción de matriz para la bolsa de palabras
    BOW = np.zeros((len(tr_text), len(vocabulary)), dtype=float) # Usamos float ya que tfidf contiene valores flotantes

    # Calculamos el IDF para cada palabra en el vocabulario
    # Necesitamos saber el número de documentos que contiene cada palabra. Realizamos esto con un diccionario
    doc_count = {word: 0 for word in dict_indices} # Inicializar diccionario
    for tr in tr_text:
        tokens = set(tokenizer.tokenize(tr))  # Convertimos a set para obtener palabras únicas
        for token in tokens:
            if token in dict_indices:  # Si la palabra está en el vocabulario
                doc_count[token] += 1

    # Iteramos sobre cada documento en tr_text
    for idx, tr in enumerate(tr_text):

        # Cada documento tr lo tokenizamos
        tokens = nltk.FreqDist(tokenizer.tokenize(tr))

        # Calculamos la frecuencia de cada palabra en el documento
        fdist_doc = nltk.FreqDist(tokens)

        # Iteramos sobre cada palabra y su frecuencia en el documento
        for word, freq in fdist_doc.items():
            if word in dict_indices:
                tf = freq / len(tokens)  # TF: Frecuencia de la palabra / total de palabras en el documento
                idf = np.log(len(tr_text) / doc_count[word])  # IDF: log(Total de documentos / úmero de documentos que contienen la palabra)
                BOW[idx, dict_indices[word]] = tf * idf

    return BOW

In [11]:
BOW_tr = build_bow_tfidf(tr_text, voc, dict_indices)
BOW_tr[:10][:10]

array([[0.        , 0.07149705, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.08579646, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.05046851, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04289823, 0.04530089, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.06471556, ..., 0.        , 0.        ,
        0.        ]])

### 4. Evalué BoW con pesado binario normalizado l2 (no use sklearn).

La normalización L2, ajusta cada vector de características (en este caso, cada fila en la matriz BoW) para que su norma (o longitud) sea 1. Esto se hace dividiendo cada elemento en el vector por la raíz cuadrada de la suma de los cuadrados de todos los elementos en el vector.

In [12]:
def build_bow_binary_norm(tr_text, vocabulary, dict_indices):
    # Construcción de matriz para la bolsa de palabras
    # En cada fila vemos los documentos que estamos procesando
    # En las columnas el tamaño del vocabulario que estamos creando
    BOW = np.zeros((len(tr_text),len(vocabulary)), dtype = float)

    for idx, tr in enumerate(tr_text):

        # Cada documento tr lo tokenizamos
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))

        # Contamos cada palabra
        for word in fdist_doc:
            # Nos aseguramos que las palabras estan en el diccionario final
            if word in dict_indices:
                BOW[idx, dict_indices[word]] = 1 # Esquema de pesado binario

    # Normalización L2 de la matriz BoW
    norm = np.linalg.norm(BOW, axis=1, keepdims=True)  # Calcular la norma L2 para cada fila (documento)
    norm[norm == 0] = 1e-10  # Asegurar que no haya divisiones por cero
    BOW = BOW / norm  # Dividir cada elemento por la norma de su fila

    return BOW

In [13]:
BOW_tr = build_bow_binary_norm(tr_text, voc, dict_indices)
BOW_tr[:10][:10]

array([[0.        , 0.20412415, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.31622777, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.24253563, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.23570226, 0.23570226, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.30151134, ..., 0.        , 0.        ,
        0.        ]])

### 5. Evalué BoW con pesado frecuencia normalizado l2 (no use sklearn).

Para normalizar, hacemos lo mismo que hicimos con el esquema binario.

In [14]:
def build_bow_frecuency_norm(tr_text, vocabulary, dict_indices):
    # Construcción de matriz para la bolsa de palabras
    BOW = np.zeros((len(tr_text), len(vocabulary)), dtype=float)

    # Iteramos sobre cada documento en tr_text
    for idx, tr in enumerate(tr_text):

        # Cada documento tr lo tokenizamos
        tokens = nltk.FreqDist(tokenizer.tokenize(tr))

        # Calculamos la frecuencia de cada palabra en el documento
        fdist_doc = nltk.FreqDist(tokens)

        # Iteramos sobre cada palabra y su frecuencia en el documento
        for word, freq in fdist_doc.items():
            # Nos aseguramos que las palabras estan en el diccionario final
            if word in dict_indices:
                BOW[idx, dict_indices[word]] = freq  # Esquema de pesado de frecuencias

    # Normalización L2 de la matriz BoW
    norm = np.linalg.norm(BOW, axis=1, keepdims=True)  # Calcular la norma L2 para cada fila (documento)
    norm[norm == 0] = 1e-10  # Asegurar que no haya divisiones por cero
    BOW = BOW / norm  # Dividir cada elemento por la norma de su fila

    return BOW

In [15]:
BOW_tr = build_bow_frecuency_norm(tr_text, voc, dict_indices)
BOW_tr[:10][:10]

array([[0.        , 0.32444284, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.31622777, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.2236068 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.21821789, 0.21821789, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.18569534, ..., 0.        , 0.        ,
        0.        ]])

### 6. Evalué BoW con pesado tfidf normalizado l2 (no use sklearn).

In [16]:
def build_bow_tfidf_norm(tr_text, vocabulary, dict_indices):
    # Construcción de matriz para la bolsa de palabras
    BOW = np.zeros((len(tr_text), len(vocabulary)), dtype=float) # Usamos float ya que tfidf contiene valores flotantes

    # Calculamos el IDF para cada palabra en el vocabulario
    # Necesitamos saber el número de documentos que contiene cada palabra. Realizamos esto con un diccionario
    doc_count = {word: 0 for word in dict_indices} # Inicializar diccionario
    for tr in tr_text:
        tokens = set(tokenizer.tokenize(tr))  # Convertimos a set para obtener palabras únicas
        for token in tokens:
            if token in dict_indices:  # Si la palabra está en el vocabulario
                doc_count[token] += 1

    # Iteramos sobre cada documento en tr_text
    for idx, tr in enumerate(tr_text):

        # Cada documento tr lo tokenizamos
        tokens = nltk.FreqDist(tokenizer.tokenize(tr))

        # Calculamos la frecuencia de cada palabra en el documento
        fdist_doc = nltk.FreqDist(tokens)

        # Iteramos sobre cada palabra y su frecuencia en el documento
        for word, freq in fdist_doc.items():
            if word in dict_indices:
                tf = freq / len(tokens)  # TF: Frecuencia de la palabra / total de palabras en el documento
                idf = np.log(len(tr_text) / doc_count[word])  # IDF: log(Total de documentos / úmero de documentos que contienen la palabra)
                BOW[idx, dict_indices[word]] = tf * idf
    
    # Normalización L2 de la matriz BoW
    norm = np.linalg.norm(BOW, axis=1, keepdims=True)  # Calcular la norma L2 para cada fila (documento)
    norm[norm == 0] = 1e-10  # Asegurar que no haya divisiones por cero
    BOW = BOW / norm  # Dividir cada elemento por la norma de su fila

    return BOW

In [17]:
BOW_tr = build_bow_tfidf_norm(tr_text, voc, dict_indices)
BOW_tr[:10][:10]

array([[0.        , 0.06525231, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.06492909, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.04781418, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.05435931, 0.05740389, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.06506392, ..., 0.        , 0.        ,
        0.        ]])

### 7. Ponga una tabla comparativa a modo de resumen con las seis entradas anteriores.

Con las funciones construidas, ahora necesitamos clasificar las matrices. Para esto podemos construir una función que realice la clasificación y llamarla las veces que sea necesario. Para implementar la clasificación haremos uso de la libería `Scikit-learn` (importado generalmente como `sklearn`) la cual es una biblioteca de Python muy popular para machine learning. Ofrece una amplia gama de algoritmos tanto para aprendizaje supervisado (como clasificación y regresión) como no supervisado (como agrupamiento y reducción de dimensionalidad), junto con herramientas para la selección de modelos, preprocesamiento de datos, evaluación de modelos y muchas otras utilidades.

In [18]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_fscore_support, roc_auc_score

In [19]:
import warnings

# Suprimir advertencias
warnings.filterwarnings('ignore')

# Función para clasificar una bolsa de palabras
def clasificar_bow(BOW_tr, tr_labels, BOW_val, val_labels):

    parameters = {'C': [0.05, .12, .25, .5, 1, 2, 4]}
    
    # Máquina de soporte vectorial con balance de clases
    srv = svm.LinearSVC(class_weight='balanced', dual=False, max_iter=10000)
    grid = GridSearchCV(estimator=srv, param_grid=parameters, n_jobs=8, scoring="f1_macro", cv=5)

    # Entrenamiento y búsqueda de hiperparámetros
    grid.fit(BOW_tr, tr_labels)

    # Predicciones sobre el conjunto de validación
    y_pred = grid.predict(BOW_val)

    # Cálculo de métricas
    precision, recall, f1_score, _ = precision_recall_fscore_support(val_labels, y_pred, average="macro")

    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1_score:.4f}")

    return precision, recall, f1_score


In [20]:
import pandas as pd

# Diccionario para almacenar los resultados
resultados = {}

# Lista de funciones de BoW
funciones_bow = [
    build_bow_binary,
    build_bow_frecuency,
    build_bow_tfidf,
    build_bow_binary_norm,
    build_bow_frecuency_norm,
    build_bow_tfidf_norm
]

# Nombres para cada función de BoW
nombres_bow = [
    "Binario",
    "Frecuencia",
    "TF-IDF",
    "Binario Normalizado",
    "Frecuencia Normalizada",
    "TF-IDF Normalizado"
]

for func, nombre in zip(funciones_bow, nombres_bow):
    print(f"Procesando: {nombre}")
    # Construir la bolsa de palabras
    BOW_tr = func(tr_text, voc, dict_indices)
    BOW_val = func(val_text, voc, dict_indices)

    # Clasificar y obtener métricas
    precision, recall, f1 = clasificar_bow(BOW_tr, tr_labels, BOW_val, val_labels)

    # Guardar resultados
    resultados[nombre] = {"Precision": precision, "Recall": recall, "F1-Score": f1}

# Convertir los resultados en un DataFrame de pandas para una visualización bonita
resultados_df = pd.DataFrame(resultados).T  # .T para transponer el DataFrame
print(resultados_df)

Procesando: Binario
Precision: 0.7692
Recall: 0.7809
F1-Score: 0.7744
Procesando: Frecuencia
Precision: 0.7892
Recall: 0.7999
F1-Score: 0.7941
Procesando: TF-IDF
Precision: 0.7932
Recall: 0.8040
F1-Score: 0.7982
Procesando: Binario Normalizado
Precision: 0.7732
Recall: 0.7868
F1-Score: 0.7792
Procesando: Frecuencia Normalizada
Precision: 0.7950
Recall: 0.8158
F1-Score: 0.8037
Procesando: TF-IDF Normalizado
Precision: 0.7818
Recall: 0.8057
F1-Score: 0.7913
                        Precision    Recall  F1-Score
Binario                  0.769177  0.780867  0.774444
Frecuencia               0.789162  0.799878  0.794089
TF-IDF                   0.793168  0.804033  0.798166
Binario Normalizado      0.773209  0.786784  0.779227
Frecuencia Normalizada   0.794952  0.815804  0.803692
TF-IDF Normalizado       0.781761  0.805668  0.791269


### 8. De las configuraciones anteriores elija la mejor y evalúela con más y menos términos
(e.g., 1000 y 7000). Ponga una tabla dónde compare las tres configuraciones.

---

Para implementar esto necesitamos construir vocabularios de distinta longitud. Las tres funciones que tuvieron mejor precisión fueron: TFIDF, Frecuencia normalizada, y frecuencia. Sin embargo vamos a probar el TFIDF, Frecuencia normalizada y TFIDF Normalizada, debido al al to valor del recall. 

Ya tenemos construido y tokenizado el corpus completo en la lista **fdist**, simplemente construimos nuevos vocabularios y llamamos a las funciones. 

In [21]:
# Función que construye un vocabulario y su diccionario de indices de diferente longitud en base a un corpus
def BuildVocabulary(fdist, Length):
    vocName = SortFrecuency(fdist)
    vocName = vocName[:Length]

    dict_indices = dict()
    count = 0

    for weight, word in vocName:
        dict_indices[word] = count
        count += 1

    return vocName, dict_indices

In [22]:
# Construimos vocabularios y diccionarios de diferente longitud
voc10mil, dict_indices10mil = BuildVocabulary(fdist, 10000)
voc7mil, dict_indices7mil = BuildVocabulary(fdist, 7000)
voc2mil, dict_indices2mil = BuildVocabulary(fdist, 2000)

Ahora llamamos a la función que clasifica las bolsas de palabras e imprimimos los resultados.

In [23]:
# Lista con los vocabularios
vocs = [voc2mil, voc7mil, voc10mil]

# Lista con los diccionarios de índices
diccionarios_indices = [dict_indices2mil, dict_indices7mil, dict_indices10mil]

In [24]:
# Lista de funciones de BoW
funciones_bow = [
    build_bow_tfidf,
    build_bow_frecuency_norm,
    build_bow_tfidf_norm
]

# Nombres para cada función de BoW
nombres_bow = [
    "TF-IDF",
    "Frecuencia Normalizada",
    "TF-IDF Normalizado"
]

# Diccionario para almacenar los resultados
resultados = {}

for voc, dict_indices in zip(vocs, diccionarios_indices):  # Usar zip para iterar en paralelo
    voc_size = len(voc)  # Obtener el tamaño del vocabulario actual
    
    for func, nombre in zip(funciones_bow, nombres_bow):
        bow_name = f"{nombre} - {voc_size} palabras"  # Crear un nombre único que incluya el tamaño del vocabulario
        print(f"Procesando: {bow_name}")
        
        # Construir la bolsa de palabras
        BOW_tr = func(tr_text, voc, dict_indices)
        BOW_val = func(val_text, voc, dict_indices)

        # Clasificar y obtener métricas
        precision, recall, f1 = clasificar_bow(BOW_tr, tr_labels, BOW_val, val_labels)

        # Guardar resultados, incluyendo el tamaño del vocabulario en la clave
        resultados[bow_name] = {"Precision": precision, "Recall": recall, "F1-Score": f1}
    

# Convertir los resultados en un DataFrame de pandas para una visualización bonita
resultados_df = pd.DataFrame(resultados).T  # .T para transponer el DataFrame
print(resultados_df)

Procesando: TF-IDF - 2000 palabras
Precision: 0.7818
Recall: 0.8057
F1-Score: 0.7913
Procesando: Frecuencia Normalizada - 2000 palabras
Precision: 0.7840
Recall: 0.8104
F1-Score: 0.7942
Procesando: TF-IDF Normalizado - 2000 palabras
Precision: 0.7802
Recall: 0.8062
F1-Score: 0.7902
Procesando: TF-IDF - 7000 palabras
Precision: 0.7932
Recall: 0.8040
F1-Score: 0.7982
Procesando: Frecuencia Normalizada - 7000 palabras
Precision: 0.7949
Recall: 0.8140
F1-Score: 0.8031
Procesando: TF-IDF Normalizado - 7000 palabras
Precision: 0.7910
Recall: 0.8081
F1-Score: 0.7984
Procesando: TF-IDF - 10000 palabras
Precision: 0.7956
Recall: 0.8017
F1-Score: 0.7985
Procesando: Frecuencia Normalizada - 10000 palabras
Precision: 0.7950
Recall: 0.8158
F1-Score: 0.8037
Procesando: TF-IDF Normalizado - 10000 palabras
Precision: 0.7816
Recall: 0.8039
F1-Score: 0.7906
                                         Precision    Recall  F1-Score
TF-IDF - 2000 palabras                    0.781761  0.805668  0.791269
Frecue

### 9. Utilice el recurso léxico del Consejo Nacional de Investigación de Canadá "EmoLex" 
con los tres pesados, normalize cada uno si lo cree conveniente.
Para construir una "Bolsa de Emociones" de los Tweets de agresividad (Debe usar EmoLex
en Español). Para esto, una estrategia sencilla sería enmascarar cada palabra con su
emoción, y después construir la Bolsa de Emociones (BoE).

### 10. Evalúa tú BoE clasificando con SVM. 
Ponga una tabla comparativa a modo de resumen

---

La Bolsa de Emociones es similar a la Bolsa de Palabras (BoW) en el procesamiento de lenguaje natural, pero en lugar de contar la frecuencia de cada palabra en un documento, cuenta la presencia o frecuencia de emociones asociadas a las palabras del documento. En el contexto de los tweets, cada tweet se transforma en un vector donde cada dimensión representa una emoción específica (como alegría, tristeza, ira, etc.), y el valor en esa dimensión puede ser binario (indicando la presencia/ausencia de esa emoción en el tweet) o una frecuencia (indicando cuántas veces se mencionan palabras asociadas a esa emoción en el tweet). Lo primero que tenemos que hacer es descargar la bolsa de emociones de EmoLex. Una vez con el diccionario descargado, lo cargamos en una variable.

In [26]:
# Importamos el archivo en un dataframe de pandas
emolex_df = pd.read_csv('/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/Spanish-NRC-EmoLex.txt', sep='\t', usecols=['Spanish Word', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust'])

# Renombrar la columna para simplificar
emolex_df.rename(columns={'Spanish Word': 'word'}, inplace=True)

# Muestra las primeras filas para verificar que se cargó correctamente
print(emolex_df.head())

   anger  anticipation  disgust  fear  joy  negative  positive  sadness  \
0      0             0        0     0    0         0         0        0   
1      0             0        0     0    0         0         0        0   
2      0             0        0     1    0         1         0        1   
3      1             0        0     1    0         1         0        1   
4      1             0        0     1    0         1         0        1   

   surprise  trust        word  
0         0      0      detrás  
1         0      1       ábaco  
2         0      0   abandonar  
3         0      0  abandonado  
4         1      0    abandono  


Ahora, para preprocesar y estructurar los datos en un formato que se pueda usar para construir una Bolsa de Emociones, se puede convertir el DataFrame en un diccionario donde cada palabra sea la clave y el valor sea otro diccionario con las emociones y sus puntuaciones binarias correspondientes.

In [27]:
emolex_dict = {}

for _, row in emolex_df.iterrows():
    word = row['word']
    # Crear un diccionario para esta palabra con cada emoción y su valor
    emotions = {emotion: row[emotion] for emotion in emolex_df.columns if emotion != 'word'}
    emolex_dict[word] = emotions

# Ejemplo de uso
print(emolex_dict.get('abandono'))

{'anger': 1, 'anticipation': 0, 'disgust': 0, 'fear': 1, 'joy': 0, 'negative': 1, 'positive': 0, 'sadness': 1, 'surprise': 1, 'trust': 0}


Ahora tenemos que "enmascarar las palabras" o cambiar cada palabra por su emoción asociada. Para enmascarar las palabras con sus emociones asociadas, se itera sobre cada palabra en los tweets, se busca su emoción asociada en el diccionario emolex_dict que creamos previamente, y se reemplaza la palabra por su emoción. Si una palabra está asociada con múltiples emociones o no se encuentra en el léxico, se puede borrar o colocar un 0. 

In [28]:
def enmascararEmociones(tr_text, emolex_dict):
    textos_enmascarados = []  # Lista para almacenar los textos transformados

    for texto in tr_text:
        # Dividir el texto en palabras
        tokens = tokenizer.tokenize(texto)
        emociones_texto = []

        for token in tokens:
            # Obtener emociones asociadas a la palabra
            emociones = emolex_dict.get(token)
            if emociones:
                # Elegir la emoción con el valor más alto o una emoción específica
                emocion_max = max(emociones, key=emociones.get)
                if emociones[emocion_max] > 0:  # Asegurarse de que la emoción tiene un valor binario positivo
                    emociones_texto.append(emocion_max)

        # Unir las emociones encontradas para formar el texto transformado
        if len(emociones_texto) > 0 : # Asegurarse que el tweet contenga emoción
            texto_enmascarado = ' '.join(emociones_texto)
            textos_enmascarados.append(texto_enmascarado)

    return textos_enmascarados  # Devolver la lista de textos transformados


In [29]:
tr_text_emociones = enmascararEmociones(tr_text, emolex_dict)
tr_text_emociones

['anticipation',
 'anticipation',
 'anger disgust',
 'anticipation anticipation joy negative negative',
 'negative',
 'anticipation',
 'anticipation anticipation',
 'anger positive',
 'trust anticipation',
 'anticipation',
 'negative positive joy anticipation joy',
 'positive negative negative',
 'anticipation anticipation trust',
 'negative fear',
 'anger trust',
 'anticipation anticipation',
 'disgust anger anticipation',
 'trust anticipation',
 'joy',
 'positive anger sadness',
 'anticipation positive',
 'negative negative',
 'trust',
 'anger',
 'negative',
 'negative',
 'positive fear',
 'joy anger anticipation',
 'negative',
 'joy',
 'anticipation positive',
 'anticipation anger',
 'fear',
 'joy anticipation',
 'anticipation',
 'anticipation joy',
 'anger positive',
 'trust',
 'anger positive',
 'anticipation',
 'negative positive',
 'negative',
 'anticipation trust anger fear',
 'anger anticipation anticipation anticipation',
 'positive fear',
 'disgust anticipation negative fear

Ahora tenemos el conjunto de tweets por emoción, ya sea principal o todas las que encuentre. Vamos a construir una bolsa de emociones con las funciones que ya realizamos para la bolsa de palabras. 

In [30]:
corpus_palabras_emociones = []

for doc in tr_text_emociones:
    corpus_palabras_emociones += tokenizer.tokenize(doc)

fdist_emociones = nltk.FreqDist(corpus_palabras_emociones)

print(f"El tamaño del corpus de emociones es:", len(corpus_palabras_emociones))
print(f"El tamaño del vocabulario de emociones es:", len(fdist_emociones))

El tamaño del corpus de emociones es: 6767
El tamaño del vocabulario de emociones es: 10


In [31]:
# Creamos un vocabulario y un diccionario
voc_emociones, dict_indices_emociones = BuildVocabulary(fdist_emociones, 10)
dict_indices_emociones

{'anticipation': 0,
 'anger': 1,
 'positive': 2,
 'negative': 3,
 'disgust': 4,
 'joy': 5,
 'trust': 6,
 'fear': 7,
 'surprise': 8,
 'sadness': 9}

Antes de poder realizar la clasificación es necesario etiquetar los tweets. En los ejemplos pasados utilizabamos el hecho de si era agresivo o no. Para las emociones podemos usar un etiquetado de emociones felices o emociones tristes. Generamos una función para etiquetar.

In [32]:
def generar_etiquetas(tr_text):
    emociones_positivas = ['joy', 'trust', 'anticipation', 'positive']
    emociones_negativas = ['sadness', 'anger', 'disgust', 'fear', 'negative']

    # Contadores para emociones positivas y negativas
    contador_positivo = 0
    contador_negativo = 0

    # Lista de etiquetas
    labels = []

    for texto in tr_text:

        # Reiniciar contadores para cada texto
        contador_positivo = 0
        contador_negativo = 0

        # Dividir el texto en palabras
        tokens = tokenizer.tokenize(texto)
        emociones_texto = []

        for token in tokens:
            # Incrementar contador si la palabra es una emoción positiva o negativa
            if token in emociones_positivas:
                contador_positivo += 1
            elif token in emociones_negativas:
                contador_negativo += 1
            
        # Etiquetar    
        if contador_positivo >= contador_negativo: # Positivo es mayor o igual ya que si no es negativo se considera positivo
            labels.append('0')  # Asumiendo que '0' indica una emoción positiva o neutra
        else:
            labels.append('1')  # Asumiendo que '1' indica una emoción negativa

    return labels

In [33]:
tr_labels_emociones = generar_etiquetas(tr_text_emociones)

# Generamos los conjuntos de validación para probar
val_text_emociones = enmascararEmociones(val_text, emolex_dict)
val_labels_emociones = generar_etiquetas(val_text_emociones)

Ahora llamamos a las funciones para construir la bolsa de emociones y posterior ejecutar un clasificador.

In [34]:
# Lista de funciones de BoW
funciones_bow = [
    build_bow_tfidf,
    build_bow_frecuency_norm,
    build_bow_tfidf_norm
]

# Nombres para cada función de BoW
nombres_bow = [
    "TF-IDF",
    "Frecuencia Normalizada",
    "TF-IDF Normalizado"
]

# Diccionario para almacenar los resultados
resultados = {}

for func, nombre in zip(funciones_bow, nombres_bow):
    print(f"Procesando: {nombre}")
    # Construir la bolsa de palabras
    BOW_tr = func(tr_text_emociones, voc_emociones, dict_indices_emociones)
    BOW_val = func(val_text_emociones, voc_emociones, dict_indices_emociones)

    # Clasificar y obtener métricas
    precision, recall, f1 = clasificar_bow(BOW_tr, tr_labels_emociones, BOW_val, val_labels_emociones)

    # Guardar resultados
    resultados[nombre] = {"Precision": precision, "Recall": recall, "F1-Score": f1}

# Convertir los resultados en un DataFrame de pandas para una visualización bonita
resultados_df = pd.DataFrame(resultados).T  # .T para transponer el DataFrame
print(resultados_df)

Procesando: TF-IDF
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000
Procesando: Frecuencia Normalizada
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000
Procesando: TF-IDF Normalizado
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000
                        Precision  Recall  F1-Score
TF-IDF                        1.0     1.0       1.0
Frecuencia Normalizada        1.0     1.0       1.0
TF-IDF Normalizado            1.0     1.0       1.0


Es posible que el modelo no este arrojando los resultados esperados, y es que valores perfectos hacen dudar acerca de si tenemos un sobre ajuste, ya que es bastante complicado tener un modelo perfecto. Una posible soución puede ser utilizar un etiquetado que sea multiclase y utilizar una clasificación multiclase, y es que estamos usando solo dos clases con una bolsa de emociones de diez emociones. Puede que no tengamos la complejidad suficiente para conseguir un modelo correcto. 

Probamos el nuevo modelo multiclase.

In [35]:
def generar_etiquetas_multiclase(tr_text):
    
    emociones_positivas = ['joy', 'trust', 'anticipation', 'positive']
    emociones_negativas = ['sadness', 'anger', 'disgust', 'fear', 'negative']
    emociones_neutrales = ['surprise']  

    labels = []

    for texto in tr_text:
        # Reiniciar contadores para cada texto
        contador_positivo = 0
        contador_negativo = 0
        contador_neutral = 0

        tokens = tokenizer.tokenize(texto)

        for token in tokens:
            if token in emociones_positivas:
                contador_positivo += 1
            elif token in emociones_negativas:
                contador_negativo += 1
            elif token in emociones_neutrales:
                contador_neutral += 1

        # Determinar la etiqueta basada en los contadores
        if contador_positivo > contador_negativo and contador_positivo > contador_neutral:
            labels.append('positivo')
        elif contador_negativo > contador_positivo and contador_negativo > contador_neutral:
            labels.append('negativo')
        elif contador_neutral > contador_positivo and contador_neutral > contador_negativo:
            labels.append('neutral')
        else:
            labels.append('mixto')  # Para textos donde no hay una emoción claramente predominante

    return labels

In [36]:
# Generamos etiquetas multiclase
tr_labels_emociones_mc = generar_etiquetas_multiclase(tr_text_emociones)
val_labels_emociones_mc = generar_etiquetas_multiclase(val_text_emociones)

Con el nuevo etiquetado, veamos el modelo.

In [37]:
# Lista de funciones de BoW
funciones_bow = [
    build_bow_tfidf,
    build_bow_frecuency_norm,
    build_bow_tfidf_norm
]

# Nombres para cada función de BoW
nombres_bow = [
    "TF-IDF",
    "Frecuencia Normalizada",
    "TF-IDF Normalizado"
]

# Diccionario para almacenar los resultados
resultados = {}

for func, nombre in zip(funciones_bow, nombres_bow):
    print(f"Procesando: {nombre}")
    # Construir la bolsa de palabras
    BOW_tr = func(tr_text_emociones, voc_emociones, dict_indices_emociones)
    BOW_val = func(val_text_emociones, voc_emociones, dict_indices_emociones)

    # Clasificar y obtener métricas
    precision, recall, f1 = clasificar_bow(BOW_tr, tr_labels_emociones_mc, BOW_val, val_labels_emociones_mc)

    # Guardar resultados
    resultados[nombre] = {"Precision": precision, "Recall": recall, "F1-Score": f1}

# Convertir los resultados en un DataFrame de pandas para una visualización bonita
resultados_df = pd.DataFrame(resultados).T  # .T para transponer el DataFrame
print(resultados_df)

Procesando: TF-IDF
Precision: 0.9909
Recall: 0.9817
F1-Score: 0.9861
Procesando: Frecuencia Normalizada
Precision: 0.8955
Recall: 0.9882
F1-Score: 0.9293
Procesando: TF-IDF Normalizado
Precision: 0.9289
Recall: 0.9893
F1-Score: 0.9546
                        Precision    Recall  F1-Score
TF-IDF                   0.990868  0.981665  0.986058
Frecuencia Normalizada   0.895504  0.988159  0.929283
TF-IDF Normalizado       0.928888  0.989305  0.954628


Con una alternativa multiclase conseguimos un mejor resultado del modelo, ya que el modelo se acerca mas a la realidad al no ser perfecto. De igual forma son resultados espectaculares para un modelo. Sería excelente ejercicio utilizar datos mas amplios para probar el modelo y verificar si los resultados siguen siendo igual de buenos. 

## Recurso Línguistico de Emociones Mexicano

### 1. Utilice el recurso léxico llamado "Spanish Emotion Lexicon (SEL)" 
Del Dr. Grigori Sidorov, profesor del Centro de Investigación en Computación (CIC) del [Instituto Politécnico Nacional](http://www.cic.ipn.mx/∼sidorov/), para enmascarar cada palabra con su emoción, y después construir la Bolsa de Emociones con algún pesado (e.g., binario, tf, tfidf).

Proponga alguna estrategia para incorporar el "valor" del "Probability Factor of Affective use" en su representación vectorial del documento. Evalúa y escribe una tabla comparativa a modo de resumen con al menos tres pesados: binario, frecuencia, tfidf. Normalize cada pesado según lo crea conveniente.

---

Necesitamos hacer el mismo proceso que el ejercicio pasado. Primero cargamos los datos en un data frame para posterior trabajar con este para construir la bolsa de emociones.

In [38]:
# Importamos el archivo en un dataframe de pandas
sel_df = pd.read_csv('/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/SEL_full.txt', sep='\t', encoding='latin-1')

# Mostrar las primeras filas del DataFrame para verificar que se cargó correctamente
print(sel_df.head())

      Palabra   Nula[%]   Baja[%]    Media[%]   Alta[%]    PFA Categoría
0  abundancia         0          0         50        50  0.830   Alegría
1    acabalar        40          0         60         0  0.396   Alegría
2     acallar        50         40         10         0  0.198   Alegría
3      acatar        50         40         10         0  0.198   Alegría
4      acción        30         30         30        10  0.397   Alegría


In [39]:
print(f"Las emociones en el data set: ", set(sel_df.Categoría))
print(f"La longitud del vocabulario: ", len(sel_df.Palabra))

Las emociones en el data set:  {'Enojo', 'Sorpresa', 'Miedo', 'Tristeza', 'Alegría', 'Repulsión'}
La longitud del vocabulario:  2036


Vamos a utilizar el conjunto de datos de MEX3T y realizaremos lo mismo que el ejercicio pasado. Construiremos una bolsa de emociones. Compararemos los modelos para ver cual nos da mejores resultados. Lo primero que tenemos que hacer es enmascarar cada valor con su emoción. Posteriormente construimos un vocabulario y luego un diccionario con el cual podamos trabajar, finalmente construimos las etiquetas y ejecutamos los modelos.

In [62]:
# Definir todas las emociones posibles
emociones_posibles = list(set(sel_df['Categoría']))

# Construir los diccionarios
sel_dict = {}
sel_dict_PFA = {}

for _, row in sel_df.iterrows():
    palabra = row['Palabra']
    categoria_sel = row['Categoría']
    PFA = row[' PFA']

    # Actualizar el diccionario de emociones para la palabra
    if palabra in sel_dict:
        sel_dict[palabra][categoria_sel] = 1
    else:
        emociones = {emocion: 0 for emocion in emociones_posibles}
        emociones[categoria_sel] = 1
        sel_dict[palabra] = emociones

    # Asignar el valor de PFA
    sel_dict_PFA[palabra] = PFA

# Ejemplo de uso para sel_dict
print(sel_dict.get('temor'))

# Ejemplo de uso para sel_dict_PFA
print(sel_dict_PFA.get('temor'))


{'Enojo': 0, 'Sorpresa': 0, 'Miedo': 1, 'Tristeza': 1, 'Alegría': 0, 'Repulsión': 0}
0.198


In [67]:
tr_text_emociones_sel = enmascararEmociones(tr_text, sel_dict)
val_text_emociones_sel = enmascararEmociones(val_text, sel_dict)

In [68]:
corpus_palabras_emociones = []

for doc in tr_text_emociones_sel:
    corpus_palabras_emociones += tokenizer.tokenize(doc)

fdist_emociones_sel = nltk.FreqDist(corpus_palabras_emociones)

print(f"El tamaño del corpus de emociones es:", len(corpus_palabras_emociones))
print(f"El tamaño del vocabulario de emociones es:", len(fdist_emociones))

El tamaño del corpus de emociones es: 1783
El tamaño del vocabulario de emociones es: 10


In [69]:
# Creamos un vocabulario y un diccionario
voc_emociones_sel, dict_indices_emociones_sel = BuildVocabulary(fdist_emociones_sel, 6)
dict_indices_emociones_sel

{'Alegría': 0,
 'Tristeza': 1,
 'Enojo': 2,
 'Sorpresa': 3,
 'Miedo': 4,
 'Repulsión': 5}

Usamos las funciones construidas. Modificamos la función que genera las etiquetas para que tenga las etiquetas en español, además agregamos el factor Probability Factor of Affectiva use, siendo positivo si encontramos que este valor predomina en la palabra. De esta manera, si el PFA supera cierto porcentaje se coloca la etiqueta positiva.

In [91]:
def generar_etiquetas_multiclase_sp(tr_text, sel_dict_PFA):
    
    emociones_positivas = ['Alegría']
    emociones_negativas = ['Enojo', 'Miedo', 'Tristeza', 'Repulsión']
    emociones_neutrales = ['Sorpresa']  

    labels = []

    for texto in tr_text:
        # Reiniciar contadores para cada texto
        contador_positivo = 0
        contador_negativo = 0
        contador_neutral = 0

        # pfa = 0

        tokens = tokenizer.tokenize(texto)

        for token in tokens:
            if token in emociones_positivas:
                contador_positivo += 1
            elif token in emociones_negativas:
                contador_negativo += 1
            elif token in emociones_neutrales:
                contador_neutral += 1

            # Calcular pfa
        #     pfa += sel_dict_PFA[token]
        
        # # Normalizar pfa 
        # pfa = pfa/len(tokens)

        # Determinar la etiqueta basada en los contadores
        if contador_positivo > contador_negativo and contador_positivo > contador_neutral:
            labels.append('positivo')
        elif contador_negativo > contador_positivo and contador_negativo > contador_neutral:
            labels.append('negativo')
        elif contador_neutral > contador_positivo and contador_neutral > contador_negativo:
            labels.append('neutral')
        else:
            labels.append('mixto')  # Para textos donde no hay una emoción claramente predominante

    return labels

In [92]:
tr_labels_emociones_sel = generar_etiquetas_multiclase_sp(tr_text_emociones_sel,sel_dict_PFA)
val_labels_emociones_sel = generar_etiquetas_multiclase_sp(val_text_emociones_sel,sel_dict_PFA)

Probamos la función

In [93]:
# Diccionario para almacenar los resultados
resultados = {}

# Lista de funciones de BoW
funciones_bow = [
    build_bow_binary,
    build_bow_frecuency,
    build_bow_tfidf
]

# Nombres para cada función de BoW
nombres_bow = [
    "Binario",
    "Frecuencia",
    "TF-IDF"
]

for func, nombre in zip(funciones_bow, nombres_bow):
    print(f"Procesando: {nombre}")
    # Construir la bolsa de palabras
    BOW_tr = func(tr_text_emociones_sel, voc_emociones_sel, dict_indices_emociones_sel)
    BOW_val = func(val_text_emociones_sel, voc_emociones_sel, dict_indices_emociones_sel)

    # Clasificar y obtener métricas
    precision, recall, f1 = clasificar_bow(BOW_tr, tr_labels_emociones_sel, BOW_val, val_labels_emociones_sel)

    # Guardar resultados
    resultados[nombre] = {"Precision": precision, "Recall": recall, "F1-Score": f1}

# Convertir los resultados en un DataFrame de pandas para una visualización bonita
resultados_df = pd.DataFrame(resultados).T  # .T para transponer el DataFrame
print(resultados_df)

Procesando: Binario


Precision: 0.9737
Recall: 0.9914
F1-Score: 0.9818
Procesando: Frecuencia
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000
Procesando: TF-IDF
Precision: 0.9952
Recall: 0.9853
F1-Score: 0.9900
            Precision    Recall  F1-Score
Binario      0.973684  0.991367  0.981756
Frecuencia   1.000000  1.000000  1.000000
TF-IDF       0.995192  0.985294  0.989997


### 2. En un comentario aparte, 
Discuta sobre la estrategía que utilizó para incorporar el "Probability Factor of Affective use". No más de 5 renglones.

---

Para incorporar el "Probability Factor of Affective use" (PFA) en la representación vectorial, utilicé el PFA como un peso al sumar las emociones en los contadores de cada categoría emocional. Al procesar cada palabra en los textos, el PFA correspondiente se sumó al contador de su categoría emocional específica. Esto permitió que las emociones con un PFA más alto tuvieran más influencia en la clasificación final del texto, reflejando así la probabilidad de uso afectivo de cada palabra y enriqueciendo la representación emocional de los textos.

## ¿Podemos mejorar con Bigramas?

### Hacer un experimento dónde concatene una buena BoW 
Según sus experimentos anteriores con otra BoW construida a partir de los 1000 bigramas más frecuentes.

--- 

Primero, necesitamos construir una bolsa de palabras buena. Vamos a usar el mismo enfoque de frecuencias.

### Hacer un experimento con las Bolsas de Emociones, Bolsa de Palabras y Bolsa de Bigramas. 
Usted elige las dimensionalidades. Para construir la representación final del documento utilice la concatenación de las representaciones según sus observaciones (e.g., Bolsa de Palabras + Bolsa de Bigramas + Bolsa de Sentimientos de Canadá + Bolsa de Sentimientos de Grigori), y aliméntelas a un SVM.

In [106]:
BOW_unigrams_tr = build_bow_frecuency(tr_text, voc, dict_indices)
BOW_unigrams_tr[:10][:10]

array([[0, 2, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

Ahora necesitamos tokenizar el texto en bigramas.

In [107]:
from nltk import bigrams
from collections import Counter
import itertools

# Tokenizamos en bigramas
tokens = [tokenizer.tokenize(texto) for texto in tr_text]
bigrams_list = list(itertools.chain(*[bigrams(token_list) for token_list in tokens]))

# Contar y obtener los 1000 bigramas más frecuentes
bigram_counts = Counter(bigrams_list)
top_1000_bigrams = [bigram for bigram, count in bigram_counts.most_common(1000)]

Ahora construimos la bow de bigramas. Definimos una funcipon para esto

In [108]:
def construir_bow_bigramas(textos, top_bigramas):
    # Inicializar una matriz donde cada fila será el vector BoW de un texto
    bow_bigramas = np.zeros((len(textos), len(top_bigramas)))

    for i, texto in enumerate(textos):
        # Tokenizar y extraer bigramas para el texto actual
        tokens = tokenizer.tokenize(texto.lower())
        bigramas_texto = list(bigrams(tokens))

        # Contar la frecuencia de cada bigrama en el texto
        frecuencias_bigramas = Counter(bigramas_texto)

        # Llenar la fila correspondiente al texto en la matriz BoW
        for j, bigrama in enumerate(top_bigramas):
            if bigrama in frecuencias_bigramas:
                bow_bigramas[i, j] = frecuencias_bigramas[bigrama]  # Frecuencia del bigrama en el texto

    return bow_bigramas

In [109]:
bow_bigramas_tr = construir_bow_bigramas(tr_text, top_1000_bigrams)
bow_bigramas_tr[:10][:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Concatenamos las matrices

In [110]:
BOW_unigrams_val = build_bow_frecuency(val_text, voc, dict_indices)
bow_bigramas_val = construir_bow_bigramas(val_text, top_1000_bigrams)

In [111]:
# Concatenar BoW de unigramas y bigramas
bow_concatenada_tr = np.hstack((BOW_unigrams_tr, bow_bigramas_tr))
bow_concatenada_val = np.hstack((BOW_unigrams_val, bow_bigramas_val))

Ahora entrenamos un modelo con esta bolsa de palabras

In [112]:
precision, recall, f1_score = clasificar_bow(bow_bigramas_tr, tr_labels, bow_bigramas_val, val_labels)

Precision: 0.6784
Recall: 0.6935
F1-Score: 0.6839


### Elabore conclusiones sobre toda esta Tarea, 
Incluyendo observaciones, comentarios y posibles mejoras futuras. Discuta el comportamiento de la BoW de usar solo palabras a integrar bigramas, y luego a integrar todo ¿ayudó? o ¿empeoró?. Discuta también
brevemente el costo computacional de los experimentos ¿Valió la Pena tener todo?. Sea
breve: todo en NO más de dos párrafos.

---

A lo largo de esta tarea, se exploraron diversas metodologías para mejorar la representación vectorial de textos mediante Bolsas de Palabras (BoW), avanzando desde unigramas hasta la integración de bigramas y la combinación de ambas representaciones. Incluir bigramas, junto con unigramas, en la BoW mostró una capacidad para capturar contextos que los unigramas por sí solos podrían no captar, como frases con significados específicos. Esta riqueza adicional en la representación de los textos puede ayudar a mejorar la precisión de los modelos de clasificación al proporcionarles un conjunto de características más informativo y contextual.

Sin embargo, la expansión de la BoW para incluir bigramas y la posterior concatenación con unigramas aumenta significativamente la dimensionalidad del espacio, lo que conlleva un mayor costo computacional tanto en términos de memoria como de tiempo de procesamiento. Este aumento en la complejidad puede no siempre justificarse por mejoras en el rendimiento del modelo, especialmente si se dispone de un conjunto de datos limitado (como en este caso). Por lo tanto, es crucial realizar una evaluación cuidadosa entre la complejidad computacional y el rendimiento del modelo. 